In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

from src.config import VARIANTS, DAILY_INSTALLS
print(VARIANTS.keys(), DAILY_INSTALLS)

dict_keys(['A', 'B']) 20000
